## Prompting Pipeline for comparison of different prompting techniques

In [ ]:
import pandas as pd
import numpy as np
import pickle
import mlflow
import mlflow.pyfunc
import sacrebleu
from sacrebleu import corpus_bleu
import hashlib
from llama_cpp import Llama
from sympy import false

### Data Loading

In [ ]:
data = pd.read_pickle("machine_translation.pkl")
data

In [ ]:
data_info = pd.DataFrame()
data_info['complexity'] = data['complexity']
data_info['text_german_length'] = data['text_german'].str.len()
data_info['text_english_length'] = data['text_english'].str.len()
data_info

### Model Loading

In [ ]:
# Modelle laden
gemma = Llama.from_pretrained(
	repo_id="lmstudio-ai/gemma-2b-it-GGUF",
	filename="gemma-2b-it-q8_0.gguf",
    n_gpu_layers=1,
    verbose=False,
)

llama32 = Llama.from_pretrained(
	repo_id="hugging-quants/Llama-3.2-3B-Instruct-Q8_0-GGUF",
	filename="llama-3.2-3b-instruct-q8_0.gguf",
    n_gpu_layers=1,
    verbose=False,
)

llama31 = Llama.from_pretrained(
	repo_id="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
	filename="Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf",
    verbose=False,
)

aya23 = Llama.from_pretrained(
	repo_id="bartowski/aya-23-35B-GGUF",
	filename="aya-23-35B-Q5_K_M.gguf",
    n_gpu_layers=1,
    verbose=False,
)

MODELS = {
    "gemma": gemma,
}

### Prompt Composition

In [ ]:
# TODO: Beide Richtungen abbilden: English <-> German
# TODO: Verschiedene Prompt Arten: zero-shot, few-shot und verschiedene Variationen reinbringen

PROMPT_TEMPLATES_ENGLISH_GERMAN = {
    "zero_shot-english": "Please translate the following text from English to German: {text}",
    #"zero_shot-german": f"Bitte übersetze den folgenden Text von Englisch nach Deutsch: {text}",
    #"few-shot-english-1": f"""Please translate a text from English to German.
    #Here are some examples:
    #- English: "Hello" -> German: "Hallo"
    #- English: "Goodbye" -> German: "Auf Wiedersehen"
    #Now translate this text: {text}""",
    #"few-shot-german-1": f"""Bitte übersetze einen Text von Englisch nach Deutsch.
    #Hier sind einige Beispiele:
    #- Englisch: "Hello" -> Deutsch: "Hallo"
    #- Englisch: "Goodbye" -> Deutsch: "Auf Wiedersehen"
    #Jetzt übersetze diesen Text: {text}""",
}

PROMPT_TEMPLATES_GERMAN_ENGLISH = {
    "zero_shot-english": "Please translate the following text from German to English: {text}",
    #"zero_shot-german": f"Bitte übersetze den folgenden Text von Deutsch nach Englisch: {text}",
    #"few-shot-english-1": f"""Please translate a text from German to English.
    #Here are some examples:
    #- German: "Hallo" -> English: "Hello"
    #- German: "Auf Wiedersehen" -> English: "Goodbye"
    #Now translate this text: {text}""",
    #"few-shot-german-1": f"""Bitte übersetze einen Text von Deutsch nach Englisch.
    #Hier sind einige Beispiele:
    #- Deutsch: "Hallo" -> Englisch: "Hello"
    #- Deutsch: "Auf Wiedersehen" -> Englisch: "Goodbye"
    #Jetzt übersetze diesen Text: {text}""",
}

### Model Interaction

In [87]:
def translate(model, prompt):
    response = model(prompt, max_tokens=512, echo=False, stop=["Q:"])
    return response['choices'][0]['text']

### Evaluation

In [88]:
def evaluate_translation(reference, translation):
    bleu_score = corpus_bleu([translation], [[reference]]).score
    return {"BLEU": bleu_score}

In [89]:
def log_to_mlflow(experiment_name, metrics, prompt_type, model_name, complexity):
    experiment = mlflow.get_experiment_by_name(experiment_name)

    if experiment:
        if experiment.lifecycle_stage == "deleted":
            mlflow.tracking.MlflowClient().restore_experiment(experiment.experiment_id)
            #mlflow.delete_experiment(experiment.experiment_id)
    else:
        mlflow.create_experiment(experiment_name)

    mlflow.set_experiment(experiment_name)
    with mlflow.start_run(run_name=f"{model_name}/{complexity}/{prompt_type}"):
        mlflow.log_param("model", model_name)
        mlflow.log_param("complexity", complexity)
        mlflow.log_param("prompt_type", prompt_type)
        for key, value in metrics.items():
            mlflow.log_metric(key, value)

### Main Pipeline Method

In [90]:
def run_pipeline(texts):
    results = pd.DataFrame(columns=["model", "complexity", "prompt_type", "prompt", "source_text", "translation", "metrics"])
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

    for model_name, model in MODELS.items():
        for _, row in texts.iterrows():

            # Übersetzung Deutsch -> Englisch
            for prompt_type, template in PROMPT_TEMPLATES_GERMAN_ENGLISH.items():
                complexity = row['complexity']
                if pd.notna(row['text_german']):
                    prompt = template.format(text=row['text_german'])
                    translation = translate(model, prompt)
                    metrics = evaluate_translation(reference=row['text_english'], translation=translation)

                    # MLflow-Logging
                    experiment_name = f"{model_name}_{complexity}"
                    log_to_mlflow(experiment_name, metrics, prompt_type, model_name, complexity)

                    # Ergebnis speichern
                    results = pd.concat([
                        results,
                        pd.DataFrame([{
                            "model": model_name,
                            "complexity": complexity,
                            "prompt_type": prompt_type,
                            "prompt": prompt,
                            "source_text": row['text_german'],
                            "translation": translation,
                            "metrics": metrics
                        }])
                    ], ignore_index=True)

                    #results.to_csv("results.csv", index=false)
                    #mlflow.log_artifact("results.csv")

            # Übersetzung Englisch -> Deutsch
            for prompt_type, template in PROMPT_TEMPLATES_ENGLISH_GERMAN.items():
                complexity = row['complexity']
                if pd.notna(row['text_english']):
                    prompt = template.format(text=row['text_english'])
                    translation = translate(model, prompt)
                    metrics = evaluate_translation(reference=row['text_german'], translation=translation)

                    # MLflow-Logging
                    experiment_name = f"{model_name}_{complexity}"
                    log_to_mlflow(experiment_name, metrics, prompt_type, model_name, complexity)

                    # Ergebnis speichern
                    results = pd.concat([
                        results,
                        pd.DataFrame([{
                            "model": model_name,
                            "complexity": complexity,
                            "prompt_type": prompt_type,
                            "prompt": prompt,
                            "source_text": row['text_english'],
                            "translation": translation,
                            "metrics": metrics
                        }])
                    ], ignore_index=True)

                    #results.to_csv("results.csv", index=false)
                    #mlflow.log_artifact("results.csv")

    return results

### Execute Pipeline

In [91]:
translation_results = run_pipeline(data)
translation_results.to_csv("translation_results.csv", index=false)
print("Pipeline abgeschlossen. Ergebnisse gespeichert.")

🏃 View run gemma/easy/zero_shot-english at: http://127.0.0.1:5000/#/experiments/689857147152224931/runs/d04335690e6340d5a7b911e5857eb790
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/689857147152224931
🏃 View run gemma/easy/zero_shot-english at: http://127.0.0.1:5000/#/experiments/689857147152224931/runs/a28fde2641af4f9dbd0dd108ff7b5197
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/689857147152224931
🏃 View run gemma/news_gen/zero_shot-english at: http://127.0.0.1:5000/#/experiments/795099840311881290/runs/66e672cbc79742cfb9a5cdb9406e90bc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/795099840311881290
🏃 View run gemma/news_gen/zero_shot-english at: http://127.0.0.1:5000/#/experiments/795099840311881290/runs/4db494f07f9945a2b6446e0685e4c323
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/795099840311881290
🏃 View run gemma/news_spec/zero_shot-english at: http://127.0.0.1:5000/#/experiments/484847269906363017/runs/210d1afdd3b1496fabde5cba346

In [93]:
translation_results

,model,complexity,prompt_type,prompt,source_text,translation,metrics
0,gemma,easy,zero_shot-english,Please translate the following text from Germa...,Felix hat es satt: Ständig ist Mama unterwegs....,,{'BLEU': 0.0}
1,gemma,easy,zero_shot-english,Please translate the following text from Engli...,Felix is fed up: Mom is always on the go. But ...,"\n\nIch hoffe, diese Übersetzung ist wie Sie g...",{'BLEU': 0.0020445979097311775}
2,gemma,news_gen,zero_shot-english,Please translate the following text from Germa...,Die rund 1.400 eingesetzten Beamten haben demn...,\n\n**Translation:**\n\nApproximately 1.400 em...,{'BLEU': 14.526747499125761}
3,gemma,news_gen,zero_shot-english,Please translate the following text from Engli...,"The approximately 1,400 deployed officers have...",\n\nPlease translate the following text from E...,{'BLEU': 0.7860561859628806}
4,gemma,news_spec,zero_shot-english,Please translate the following text from Germa...,"Der Staatschef hat zugleich aber das Recht, vo...",,{'BLEU': 0.0}
5,gemma,news_spec,zero_shot-english,Please translate the following text from Engli...,The head of state also has the right to appoin...,This could potentially affect the government'...,{'BLEU': 0.6057232211712502}
6,gemma,pop_science,zero_shot-english,Please translate the following text from Germa...,Dass der Klimawandel die Hitzewellen in Südasi...,"\n\nIn dieser Zeit ist es wichtig, die Rolle v...",{'BLEU': 1.9084912671852643}
7,gemma,pop_science,zero_shot-english,Please translate the following text from Engli...,There is no question that climate change is in...,\n\nThe consensus among experts is clear: clim...,{'BLEU': 0.11276930404887484}
8,gemma,science,zero_shot-english,Please translate the following text from Germa...,"Der DSA-110, der sich am Owens Valley Radio Ob...",\n\n**English Translation:**\n\nThe DSA-110 ra...,{'BLEU': 41.601423151567076}
9,gemma,science,zero_shot-english,Please translate the following text from Engli...,"The DSA-110, situated at the Owens Valley Radi...",\n\n**Questions:**\n1. In which observatory is...,{'BLEU': 3.5886176944270933}


In [85]:
mlflow.end_run()